In [1]:
#tải các thư viện cần thiết
!pip install transformer
!pip install seqeval

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=d2fede19fad9fab4f3f937ef0cd058c6ea29b0897924f3e9a5058dceacc9ac05
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


# **Thư viện**

In [2]:
#import các thư viện cần thiết
import numpy as np
import pandas as pd
import torch
import warnings
warnings.filterwarnings('ignore')

#Embedding
from transformers import AutoModel, AutoTokenizer, AutoModelForTokenClassification

#Modeling
from torch.utils.data import DataLoader
from torch.optim import SGD, Adam
from seqeval.metrics import classification_report
from tqdm import tqdm

Chuyển đổi bộ dữ liệu vlsp_2018

In [3]:
import re
import os
train_dir = os.listdir('/kaggle/input/vlsp-2018/VLSP_2018/VLSP2018-NER-train-Jan14')
test_dir = os.listdir('/kaggle/input/vlsp-2018/VLSP_2018/VLSP2018-NER-Test-Domains')
val_dir = os.listdir('/kaggle/input/vlsp-2018/VLSP_2018/VLSP2018-NER-dev')

def load_from_directory(data_dir, path, type: str):
  sentences, connl = [], []
  for i,n in enumerate(data_dir):
    with open(f'{path}/{n}','r',encoding='utf-8') as file:
      for line in file:
        line = line.strip()
        sentences.append(line)

  for sentence in sentences:
      PER = re.findall(r'<ENAMEX TYPE="PERSON">(.+?)</ENAMEX>', sentence)
      ORG = re.findall(r'<ENAMEX TYPE="ORGANIZATION">(.+?)</ENAMEX>', sentence)
      LOC = re.findall(r'<ENAMEX TYPE="LOCATION">(.+?)</ENAMEX>', sentence)
      MISC = re.findall(r'<ENAMEX TYPE="MISCELLANEOUS">(.+?)</ENAMEX>', sentence)
      splitted = re.split('<ENAMEX TYPE="PERSON">|</ENAMEX>|<ENAMEX TYPE="ORGANIZATION">|</ENAMEX>|<ENAMEX TYPE="LOCATION">|</ENAMEX>|<ENAMEX TYPE="MISCELLANEOUS">|</ENAMEX>', sentence)
      if PER or ORG or LOC:
          for split in splitted:
              if split in PER:
                  counter = 0
                  for token in split.split():
                      if counter > 0:
                          connl.append(token + ' I-PER')
                      else:
                          connl.append(token + ' B-PER')
                      counter += 1

              elif split in ORG:
                  counter = 0
                  for token in split.split():
                      if counter > 0:
                          connl.append(token + ' I-ORG')
                      else:
                          connl.append(token + ' B-ORG')
                      counter += 1

              elif split in LOC:
                  counter = 0
                  for token in split.split():
                      if counter > 0:
                          connl.append(token + ' I-LOC')
                      else:
                          connl.append(token + ' B-LOC')
                      counter += 1

              elif split in MISC:
                  counter = 0
                  for token in split.split():
                      if counter > 0:
                          connl.append(token + ' I-MISC')
                      else:
                          connl.append(token + ' B-MISC')
                      counter += 1

              else:  # Không là các thực thể trên đánh nhãn O
                  for token in split.split():
                      connl.append(token + ' O')

      else:  #Ngoài các thực thể trên sẽ đánh nhãn O
          for word in sentence.split():
              connl.append(word + ' O')

      connl.append('')

  if type == 'train':
    with open('/kaggle/working/train_set_2018.txt', 'w', encoding='utf-8') as output:
        for element in connl:
            output.write(element + "\n")
  elif type == 'test':
    with open('/kaggle/working/test_set_2018.txt', 'w', encoding='utf-8') as output:
        for element in connl:
            output.write(element + "\n")
  else:
    with open('/kaggle/working/val_set_2018.txt', 'w', encoding='utf-8') as output:
        for element in connl:
            output.write(element + "\n")
            
load_from_directory(train_dir, '/kaggle/input/vlsp-2018/VLSP_2018/VLSP2018-NER-train-Jan14','train')
load_from_directory(test_dir, '/kaggle/input/vlsp-2018/VLSP_2018/VLSP2018-NER-Test-Domains','test')
load_from_directory(val_dir, '/kaggle/input/vlsp-2018/VLSP_2018/VLSP2018-NER-dev','val')

func lấy data covidPho

In [4]:
#hàm đọc dữ liệu
def read_dataset(file_path):
    tokens=[]
    ner_tags=[]
    ids=[]
    count=1
    with open(file_path) as f:
        lines=f.readlines()
        ts=[]
        nts=[]
        for line in lines:
            line = line.split()
            if len(line)==0:
                ids.append(count)
                tokens.append(ts)
                ner_tags.append(nts)
                ts=[]
                nts=[]
                count+=1
            else:
                ts.append(line[0])
                nts.append(line[-1])
    data = pd.DataFrame({'Id':ids, 'NER_tags':ner_tags, 'Tokens':tokens})
    return data

In [5]:
cv_train = read_dataset('/kaggle/input/covid19vi/syllable/train_syllable.conll')
cv_val = read_dataset('/kaggle/input/covid19vi/syllable/dev_syllable.conll')
cv_test = read_dataset('/kaggle/input/covid19vi/syllable/test_syllable.conll')

In [6]:
cv_train

,Id,NER_tags,Tokens
0,1,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Đồng, thời, ,, bệnh, viện, tiếp, tục, thực, h..."
1,2,"[O, O, O, O, O, O, O, O, O, O, O, B-SYMPTOM_AN...","["", Số, bệnh, viện, có, thể, tiếp, nhận, bệnh,..."
2,3,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Ngoài, ra, ,, những, người, tiếp, xúc, gián, ..."
3,4,"[O, O, O, O, O, O, O, B-LOCATION, O, B-LOCATIO...","[Bà, này, khi, trở, về, quá, cảnh, Doha, (, Qa..."
4,5,"[O, O, O, B-PATIENT_ID, O, O, O, O, O, O, O, B...","["", Bệnh, nhân, 523, "", và, chồng, là, "", bệnh..."
...,...,...,...
5022,5023,"[O, O, O, B-LOCATION, I-LOCATION, I-LOCATION, ...","[Liên, quan, đến, Bệnh, viện, Bạch, Mai, ,, ôn..."
5023,5024,"[O, O, O, O, B-DATE, O, O, O, O, O, O, O]","[Mẫu, lần, hai, ngày, 22/7, kết, quả, sàng, lọ..."
5024,5025,"[O, O, O, O, O, O, O, O, B-ORGANIZATION, I-ORG...","[Đây, là, 5, trường, hợp, dương, tính, được, B..."
5025,5026,"[O, O, O, B-DATE, I-DATE, I-DATE, O, B-ORGANIZ...","[Lúc, 17h, ngày, 7, -, 3, ,, Viện, Vệ, sinh, D..."


Hàm đọc dữ liệu

In [7]:
def read_dataset(file_path):
    tokens=[]
    ner_tags=[]
    ids=[]
    count=1
    with open(file_path) as f:
        lines=f.readlines()
        ts=[]
        nts=[]
        for line in lines:
            line = line.split()
            if len(line)==0:
                if len(ts) > 0:  # Kiểm tra xem danh sách ts có dữ liệu không trước khi thêm vào
                    ids.append(count)
                    tokens.append(ts)
                    ner_tags.append(nts)
                    ts=[]
                    nts=[]
                    count+=1
            else:
                ts.append(line[0])
                nts.append(line[-1])
    if len(ts) > 0:  # Kiểm tra xem danh sách ts có dữ liệu không sau khi kết thúc vòng lặp
        ids.append(count)
        tokens.append(ts)
        ner_tags.append(nts)
    data = pd.DataFrame({'Id':ids, 'NER_tags':ner_tags, 'Tokens':tokens})
    return data


In [8]:
vlsp_train = read_dataset('/kaggle/working/train_set_2018.txt')
vlsp_val = read_dataset('/kaggle/working/val_set_2018.txt')
vlsp_test = read_dataset('/kaggle/working/test_set_2018.txt')

In [9]:
vlsp_train

,Id,NER_tags,Tokens
0,1,"[O, O, O, O, B-ORG, O, O, B-LOC, I-LOC, O, O, ...","[Phung, phí, cơ, hội,, SLNA, và, B., Bình, Dươ..."
1,2,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Chơi, phòng, ngự, kín, kẽ, nhưng, hạn, chế, v..."
2,3,"[B-ORG, I-ORG, O, O, B-PER, I-PER, I-PER, O, B...","[Bình, Dương, :, TM, Bùi, Tấn, Trường, ,, Mich..."
3,4,"[O, O, B-PER, I-PER, O, B-PER, I-PER, O, B-PER...","[SLNA:, TM, Nguyên, Mạnh, ,, Ngọc, Hải, ,, Văn..."
4,5,"[O, O, O, B-ORG, O, O, O, O, O, O, O, O, O, O,...","[Những, phút, đầu,, SLNA, nhập, cuộc, khá, thậ..."
...,...,...,...
9330,9331,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Lễ, hội, Trung, thu, năm, nay, diễn, ra, tron..."
9331,9332,"[O, O, O, O, O, O, O, B-LOC, I-LOC, I-LOC, O, ...","[Trong, thời, gian, diễn, ra, lễ, hội,, quận, ..."
9332,9333,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Bên, cạnh, đó,, các, hoạt, động, văn, hóa, co..."
9333,9334,"[O, O, O, O, B-LOC, I-LOC, I-LOC, O, O, O, O, ...","[Tại, lễ, khai, mạc,, quận, Hoàn, Kiếm, đã, tr..."


hàm đọc nhãn NER

In [10]:
def get_NER_labels(data):
    NERs = list(data['NER_tags'].values)
    labels_list = []
    for value in NERs:
        labels_list = labels_list + value
    types = list(set(labels_list))
    return types

In [11]:
cv_labels = get_NER_labels(cv_train)
cv_labels

['I-AGE',
 'I-ORGANIZATION',
 'B-GENDER',
 'B-PATIENT_ID',
 'B-TRANSPORTATION',
 'I-SYMPTOM_AND_DISEASE',
 'I-NAME',
 'B-LOCATION',
 'I-TRANSPORTATION',
 'I-PATIENT_ID',
 'B-SYMPTOM_AND_DISEASE',
 'B-JOB',
 'B-ORGANIZATION',
 'I-LOCATION',
 'B-DATE',
 'B-AGE',
 'I-DATE',
 'I-GENDER',
 'I-JOB',
 'O',
 'B-NAME']

In [12]:
translation_dict = {
    'I-NAME': 'I-PER',
    'B-NAME': 'B-PER',
    'I-LOCATION': 'I-LOC',
    'B-LOCATION': 'B-LOC',
    'I-ORGANIZATION': 'I-ORG',
    'B-ORGANIZATION': 'B-ORG'
}

In [13]:
def change_labels(row):
    new_row = [translation_dict[label] if label in translation_dict else label for label in row]
    return new_row
cv_train['NER_tags'] = cv_train['NER_tags'].apply(change_labels)
cv_val['NER_tags'] = cv_val['NER_tags'].apply(change_labels)
cv_test['NER_tags'] = cv_test['NER_tags'].apply(change_labels)
# vlsp_train['NER_tags'] = vlsp_train['NER_tags'].apply(change_labels)
# vlsp_val['NER_tags'] = vlsp_val['NER_tags'].apply(change_labels)
# vlsp_test['NER_tags'] = vlsp_test['NER_tags'].apply(change_labels)

In [14]:
vlsp_labels = get_NER_labels(vlsp_train)
vlsp_labels

['B-LOC', 'I-LOC', 'I-PER', 'B-ORG', 'I-MISC', 'B-MISC', 'I-ORG', 'O', 'B-PER']

In [15]:
combined_train = pd.concat([cv_train, vlsp_train], ignore_index=True)
combined_train['Id'] = range(1, len(combined_train) + 1)

In [16]:
combined_val = pd.concat([cv_val, vlsp_val], ignore_index=True)
combined_val['Id'] = range(1, len(combined_val) + 1)

In [17]:
combined_test = pd.concat([cv_test, vlsp_test], ignore_index=True)
combined_test['Id'] = range(1, len(combined_test) + 1)

In [18]:
combined_train

,Id,NER_tags,Tokens
0,1,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Đồng, thời, ,, bệnh, viện, tiếp, tục, thực, h..."
1,2,"[O, O, O, O, O, O, O, O, O, O, O, B-SYMPTOM_AN...","["", Số, bệnh, viện, có, thể, tiếp, nhận, bệnh,..."
2,3,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Ngoài, ra, ,, những, người, tiếp, xúc, gián, ..."
3,4,"[O, O, O, O, O, O, O, B-LOC, O, B-LOC, O, O, O...","[Bà, này, khi, trở, về, quá, cảnh, Doha, (, Qa..."
4,5,"[O, O, O, B-PATIENT_ID, O, O, O, O, O, O, O, B...","["", Bệnh, nhân, 523, "", và, chồng, là, "", bệnh..."
...,...,...,...
14357,14358,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Lễ, hội, Trung, thu, năm, nay, diễn, ra, tron..."
14358,14359,"[O, O, O, O, O, O, O, B-LOC, I-LOC, I-LOC, O, ...","[Trong, thời, gian, diễn, ra, lễ, hội,, quận, ..."
14359,14360,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Bên, cạnh, đó,, các, hoạt, động, văn, hóa, co..."
14360,14361,"[O, O, O, O, B-LOC, I-LOC, I-LOC, O, O, O, O, ...","[Tại, lễ, khai, mạc,, quận, Hoàn, Kiếm, đã, tr..."


In [19]:
# # Function to convert DataFrame to .conll format
# def dataframe_to_conll(df, filepath):
#     with open(filepath, 'w', encoding='utf-8') as file:
#         for index, row in df.iterrows():
#             for token, tag in zip(row['Tokens'], row['NER_tags']):
#                 file.write(f"{token}\t{tag}\n")
#             file.write("\n")  # Separate sentences by a newline
            
# # Convert DataFrame to .conll file
# dataframe_to_conll(combined_train, '/kaggle/working/train.conll')

In [20]:
unique_labels = get_NER_labels(combined_train)
unique_labels

['I-AGE',
 'B-GENDER',
 'B-PATIENT_ID',
 'B-TRANSPORTATION',
 'I-MISC',
 'I-SYMPTOM_AND_DISEASE',
 'I-TRANSPORTATION',
 'B-ORG',
 'I-PATIENT_ID',
 'B-MISC',
 'B-SYMPTOM_AND_DISEASE',
 'B-JOB',
 'I-ORG',
 'B-PER',
 'B-LOC',
 'I-LOC',
 'I-PER',
 'B-DATE',
 'B-AGE',
 'I-DATE',
 'I-GENDER',
 'I-JOB',
 'O']

# **PhoBERT**

In [21]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

# **TẠO LỚP DATASET BẰNG PYTORCH DATASET**

In [22]:
def align_label(text, labels, flag=False):
    
    label_all_tokens = flag #flag xác định cách thực hiện align_label

    tokenized_input = tokenizer(text, padding='max_length', max_length=256, truncation=True, is_split_into_words=True)
    
    # Lấy các ID của token từ đầu vào đã được mã hóa
    word_ids = tokenized_input.input_ids
    
    start_part = True # Biến để xác định xem có phải là phần đầu của từ hay không
    label_ids = [] # Danh sách để lưu trữ các ID nhãn tương ứng
    count = 0 # Biến đếm để theo dõi vị trí trong danh sách labels
    
    for i in range(len(word_ids)): # Vòng lặp qua từng ID token trong word_ids
        
        if word_ids[i] == 0 or word_ids[i] == 1 or word_ids[i] == 2:
            label_ids.append(-100) # Nếu token là token đặc biệt (0, 1, 2), gán nhãn -100
            # Kiểm tra xem token hiện tại có khớp với từ gốc trong văn bản không
        elif count < len(text) and ''.join(tokenizer.decode(tokenized_input['input_ids'][i]).split()) == text[count]:
            label_ids.append(labels_to_ids[labels[count]]) # Gán nhãn tương ứng từ labels
            count+=1 # Tăng biến đếm
            start_part = True # Đánh dấu là phần đầu của từ
        else:
            if start_part:
                label_ids.append(labels_to_ids[labels[count]])# Gán nhãn cho phần đầu của từ
                count+=1 # Tăng biến đếm
                start_part = False # Đánh dấu không còn là phần đầu của từ
            else:
                label_ids.append(labels_to_ids[labels[count]] if label_all_tokens else -100)
                # Nếu không phải phần đầu và flag label_all_tokens là True, gán nhãn cho tất cả các token
                # Nếu flag label_all_tokens là False, gán nhãn -100 cho các token không phải phần đầu của từ
                
    return label_ids

class DataSet(torch.utils.data.Dataset):

    def __init__(self, df, flag_align_label=False):

        lb = df['NER_tags'].values.tolist()
        txt = df['Tokens'].values.tolist()
        self.texts = [tokenizer(i, padding='max_length', max_length = 256,
                                truncation=True, return_tensors="pt", is_split_into_words=True) for i in txt]
        self.labels = [align_label(i,j,flag_align_label) for i,j in zip(txt, lb)]
        
    def __len__(self):

        return len(self.labels)

    def get_data(self, idx):
        return self.texts[idx]

    def get_labels(self, idx):
        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):
        data = self.get_data(idx)
        labels = self.get_labels(idx)

        return data, labels

**Tạo 2 dictionary để xác định id nào sẽ là label nào và ngược lại.**

In [23]:
labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
ids_to_labels = {v: k for v, k in enumerate(unique_labels)}

In [24]:
labels_to_ids

{'I-AGE': 0,
 'B-GENDER': 1,
 'B-PATIENT_ID': 2,
 'B-TRANSPORTATION': 3,
 'I-MISC': 4,
 'I-SYMPTOM_AND_DISEASE': 5,
 'I-TRANSPORTATION': 6,
 'B-ORG': 7,
 'I-PATIENT_ID': 8,
 'B-MISC': 9,
 'B-SYMPTOM_AND_DISEASE': 10,
 'B-JOB': 11,
 'I-ORG': 12,
 'B-PER': 13,
 'B-LOC': 14,
 'I-LOC': 15,
 'I-PER': 16,
 'B-DATE': 17,
 'B-AGE': 18,
 'I-DATE': 19,
 'I-GENDER': 20,
 'I-JOB': 21,
 'O': 22}

In [25]:
ids_to_labels

{0: 'I-AGE',
 1: 'B-GENDER',
 2: 'B-PATIENT_ID',
 3: 'B-TRANSPORTATION',
 4: 'I-MISC',
 5: 'I-SYMPTOM_AND_DISEASE',
 6: 'I-TRANSPORTATION',
 7: 'B-ORG',
 8: 'I-PATIENT_ID',
 9: 'B-MISC',
 10: 'B-SYMPTOM_AND_DISEASE',
 11: 'B-JOB',
 12: 'I-ORG',
 13: 'B-PER',
 14: 'B-LOC',
 15: 'I-LOC',
 16: 'I-PER',
 17: 'B-DATE',
 18: 'B-AGE',
 19: 'I-DATE',
 20: 'I-GENDER',
 21: 'I-JOB',
 22: 'O'}

# **BUILD MODEL PHOBERT FOR TOKEN CLASSIFICATION**

In [26]:
class PhoBertModel(torch.nn.Module):

    def __init__(self):

        super(PhoBertModel, self).__init__()

        self.phobert = AutoModelForTokenClassification.from_pretrained("vinai/phobert-base", num_labels=len(unique_labels))

    def forward(self, input_id, mask, label):
        
        output = self.phobert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)
        
        return output

# **TRAIN MODEL**

In [27]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [28]:
def train_loop(model, train_df, val_df, flag_align_label):

    train_dataset = DataSet(train_df, flag_align_label)
    val_dataset = DataSet(val_df, flag_align_label)

    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()
        
    min_val_loss = 1000
    count = 0
    
    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)

            for i in range(logits.shape[0]):

                logits_clean = logits[i][train_label[i] != -100]
                label_clean = train_label[i][train_label[i] != -100]

                predictions = logits_clean.argmax(dim=1)
                acc = (predictions == label_clean).float().mean()
                total_acc_train += acc
                total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, val_label)

            for i in range(logits.shape[0]):

                logits_clean = logits[i][val_label[i] != -100]
                label_clean = val_label[i][val_label[i] != -100]

                predictions = logits_clean.argmax(dim=1)
                acc = (predictions == label_clean).float().mean()
                total_acc_val += acc
                total_loss_val += loss.item()

        val_accuracy = total_acc_val / len(val_df)
        val_loss = total_loss_val / len(val_df)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(train_df): .3f} | Accuracy: {total_acc_train / len(train_df): .3f} | Val_Loss: {total_loss_val / len(val_df): .3f} | Accuracy: {total_acc_val / len(val_df): .3f}')
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            torch.save(model.state_dict(), 'phobert_base_ner')
            count = epoch_num
        if epoch_num - count >= 5:
            return
        
LEARNING_RATE = 5e-5
EPOCHS = 10
BATCH_SIZE = 32

model = PhoBertModel()
train_loop(model, combined_train, combined_val, False)

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2024-05-22 16:01:49.359590: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 16:01:49.359688: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 16:01:49.479208: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
100%|██████████| 449/449 [05:36<00:00,  1.33it/s]


Epochs: 1 | Loss:  0.256 | Accuracy:  0.937 | Val_Loss:  0.088 | Accuracy:  0.978


100%|██████████| 449/449 [05:33<00:00,  1.35it/s]


Epochs: 2 | Loss:  0.057 | Accuracy:  0.985 | Val_Loss:  0.065 | Accuracy:  0.980


...

Epochs: 6 | Loss:  0.014 | Accuracy:  0.996 | Val_Loss:  0.071 | Accuracy:  0.983


100%|██████████| 449/449 [05:33<00:00,  1.34it/s]


Epochs: 7 | Loss:  0.011 | Accuracy:  0.997 | Val_Loss:  0.071 | Accuracy:  0.982


100%|██████████| 449/449 [05:34<00:00,  1.34it/s]


Epochs: 8 | Loss:  0.011 | Accuracy:  0.996 | Val_Loss:  0.073 | Accuracy:  0.980


# **EVALUATE MODEL**

In [29]:
def evaluate(model, test_df, flag_align_label, ids_to_labels):

    test_dataset = DataSet(test_df, flag_align_label)

    test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    predictions = []
    labels = []
    
    for test_data, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_data['attention_mask'].squeeze(1).to(device)

            input_id = test_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, test_label)

            for i in range(logits.shape[0]):
                cleaned_logits = logits[i][test_label[i] != -100].argmax(dim=1)
                predictions.append([ids_to_labels[val.item()] for val in cleaned_logits])
                cleaned_labels = test_label[i][test_label[i] != -100]
                labels.append([ids_to_labels[val.item()] for val in cleaned_labels])
    print(classification_report(y_pred=predictions, y_true=labels, digits=3))

evaluate(model, combine_test, False, ids_to_labels)

NameError: name 'combine_test' is not defined

# **INFERENCE**

In [ ]:
def align_word_ids(text, flag):
    label_all_tokens = flag
    
    text = text.split()
  
    tokenized_inputs = tokenizer(text, padding='max_length', max_length=256, truncation=True, is_split_into_words=True)

    word_ids = tokenized_inputs.input_ids

    start_part = True
    label_ids = []
    count = 0
    
    for i in range(len(word_ids)):
        
        if word_ids[i] == 0 or word_ids[i] == 1 or word_ids[i] == 2:
            label_ids.append(-100)
            
        elif count < len(text) and ''.join(tokenizer.decode(tokenized_inputs['input_ids'][i]).split()) == text[count]:
            label_ids.append(1)
            count+=1
            start_part = True
        else:
            if start_part:
                label_ids.append(1)
                count+=1
                start_part = False
            else:
                label_ids.append(1 if label_all_tokens else -100)           
    return label_ids

In [ ]:
def ner(model, sentence, flag_align_label):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
    text = tokenizer(sentence, padding='max_length', max_length = 256, truncation=True, return_tensors="pt")

    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.Tensor(align_word_ids(sentence, flag_align_label)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    print(sentence)
    print(prediction_label)

In [ ]:
ner(model,
    'Harry Kane là cầu thủ của câu lạc bộ Tottenham nhọ nhất nước Anh',
    flag_align_label=False)